In [3]:
from bs4 import BeautifulSoup
from pymongo import MongoClient
import requests
import json

In [4]:
url = 'https://www.rei.com/product/871413/patagonia-down-sweater-mens'

In [5]:
r = requests.get(url)
soup = BeautifulSoup(r.content, 'lxml')

In [ ]:
soup

In [ ]:
for link in lst:
    r = requests.get(link)
    soup = BeautifulSoup(r.content, 'lxml')
    wrtie to MDB
    

In [4]:
client = MongoClient()

In [5]:
db = client.products

In [6]:
collection = db.rei

Things to scrape:
- Carosel - images:
    Run through json parser and select "color: ... "
- Class product-item-details
- class product-spec-table table
- product low and high price  from "div id" <div id="js-product-information-price" 
- header <h2 data-ui="product-information-title" class="header product-information-title">
- description <p class="product-primary-description"
- 


In [13]:
def parseSpecs(spec_list):
    dic = {}
    i=0
    while i < len(spec_list)-2:
        dic[spec_list[i]] = spec_list[i+1]
        i += 2

In [18]:
def productInfo(collection, url):
    dic = {}
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    dic['soup'] = str(soup)
    dic['title'] = soup.select('title')[0].text
    dic['description'] = soup.select('p.product-primary-description')[0].text.strip()
    dic['details'] = soup.select('ul.product-item-details')[0].text.split('\n')
    dic['specs'] = parseSpecs([u.strip() for u in soup.select('table.product-spec-table')[0].text.split('\n') if u != ''])
    md = json.loads(soup.findAll('script', {'data-client-store': 'page-meta-data'})[0].text)
    dic['meta_data'] = md
    dic['average_rating'] = md['averageRating']
    dic['color_count'] = md['pdpcolornum']
    dic['gender'] = md['productGender']
    dic['review_count'] = md['reviewCount']
    dic['product_path'] = md['productCategoryPath']
    dic['color_list'] = json.loads(soup.findAll('script', {'data-client-store': 'carousel-images'})[0].text).keys()
    img_list = []
    for l in soup.findAll('img',{'class': 'product-image-thumbnail'}):
        img_list.extent(l['data-high-res-img'])
    dic['img_list'] = img_list
    collection.insert_one(dic)

In [19]:
productInfo(collection, url)

In [21]:
collection.find_one()

{u'_id': ObjectId('57f3112f7c6fb889f5bc84f8'),
 u'average_rating': 4.419354838709677,
 u'color_count': 13,
 u'color_list': [u'FEATHER GREY/FORGE GREY',
  u'ASH TAN',
  u'NAVY BLUE/RAMBLE RED',
  u'UNDERWATER BLUE/FRENCH RED',
  u'FRENCH RED',
  u'DEEP SEA BLUE',
  u'FORGE GREY/NAMBRE',
  u'BLACK',
  u'PEPPERGRASS GREEN',
  u'SUPPLY GREEN',
  u'RAMBLE RED',
  u'LEGEND GREEN',
  u'NAVY BLUE/UNDERWATER BLUE'],
 u'description': u'This down sweater delivers ultralight, compressible warmth for your cold-weather fun outside while everyone else is languishing at the gym or huddled by the fireplace.',
 u'details': [u'',
  u'100% recycled ripstop shell and lining fabric have a durable water repellent finish',
  u'Traceable Down (third-party-verified, non-live-plucked, non-force-fed) 800-fill-power European goose down is stabilized with quilted-through construction',
  u'2 exterior zippered pockets; interior chest pocket doubles as a zippered stuff sack with a carabiner clip-in loop',
  u'Comfort

In [ ]:
json.loads(soup.findAll('script', {'data-client-store': 'carousel-images'})[0].text).keys()

In [ ]:
soup.select('div.product-carousel')[0]

In [13]:
for l in soup.findAll('img',{'class': 'product-image-thumbnail'}):
    print l['data-high-res-img']

/media/be1ccefb-97dd-47b2-b189-5e3f32ef25ee?size=2000
/media/a540e99d-6530-43a5-973e-b0e8bb047edb?size=2000
/media/93c0c984-722f-4836-967c-fd14e9263a7f?size=2000
/media/2bda7b09-8aef-447c-b0e7-7f157c49cccb?size=2000
/media/0f6639a6-7fdf-4f17-834e-7bc70bb66271?size=2000
/media/6d10b3c1-c3f4-47a8-8dd6-ef2b90131f29?size=2000
/media/3fc4a2ec-700f-4938-9893-72a9471b56a3?size=2000


In [10]:
soup

<!DOCTYPE html>\n<!-- If you are passionate about the outdoors and technology, visit http://rei.jobs! --><html lang="en-us" prefix="og: http://ogp.me/ns#">\n<head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# product: http://ogp.me/ns/product#"> <!-- meta format --> <meta charset="unicode-escape"/> <meta content="telephone=no" name="format-detection"/> <meta content="IE=edge" http-equiv="X-UA-Compatible"/> <meta content="origin-when-cross-origin" name="referrer"/> <!-- meta info --> <title>Patagonia Down Sweater - Men's - REI.com</title><meta content="This down sweater delivers ultralight, compressible warmth for your cold-weather fun outside while everyone else is languishing at the gym or huddled by the fireplace. Available at REI, 100% Satisfaction Guaranteed." name="description"/> <meta content="REI" name="author"/> <meta content="\xa9 2016 Recreational Equipment, Inc. All rights reserved. REI, REI Garage and the REI co-op logo are trademarks of Recreational Equipment, Inc